# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил(а): Разуваев Никита Сергеевич

### Общая информация
Дата выдачи: 09.04.2020

Дедлайн: 24.04.2020 23:59 MSK

### О задании

В этом домашнем задании вы будете работать с линейной классификацией, попрактикуетесь на реальной задаче классификации текстов.

Для решения этого домашнего задания намного удобнее будет использовать Colab, так как данных много.

### Оценивание и штрафы

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи
Загрузка файлов с решениями происходит в системе Anytask.

### Подготовка данных

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm.notebook import tqdm

In [2]:
# Датасет можно скачать здесь

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

--2020-04-24 20:50:19--  https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tg55q9mrziroyrs/train_subset.csv [following]
--2020-04-24 20:50:20--  https://www.dropbox.com/s/raw/tg55q9mrziroyrs/train_subset.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc12cca46263216fa5d8fd63c5f5.dl.dropboxusercontent.com/cd/0/inline/A2e2n3x9g4a03vL0lw1fSJRno0Zf4JaQho6JfNN9JShsdUX8LEO6GcbT8slKGU8zTqfcU0JK8FuB-NHTqqMyfgzIqPzbSBEayz_ouj3X4CO8X8bee48RRAyAcjBt1aEdvR8/file# [following]
--2020-04-24 20:50:20--  https://uc12cca46263216fa5d8fd63c5f5.dl.dropboxusercontent.com/cd/0/inline/A2e2n3x9g4a03vL0lw1fSJRno0Zf4JaQho6JfNN9JShsdUX8LEO6GcbT8slKGU8zTqfcU0JK8FuB-NHTqqMyfgzIqPzbSBEayz_ouj

### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [3]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [4]:
data.shape

(30000, 4)

In [5]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Давайте пока остановимся на простом WordPunctTokenizer.


In [8]:
from nltk.tokenize import WordPunctTokenizer


tokenizer = WordPunctTokenizer()


def preprocess(text: str) -> str:
    return ' '.join(tokenizer.tokenize(text.lower()))


text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'
print("before:", text,)
print("after:", preprocess(text),)

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...


__Задание:__ Токенизируйте title и description в train и test

In [9]:
# Your code here
preprocess_func = np.vectorize(preprocess)
X_train = preprocess_func(X_train)

In [10]:
X_train[:10]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п .

In [11]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

### Логистическая регрессия и SVC (1 балл)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.

**Подсказка: рассмотрите использование sparse-матриц - в них хранятся только ненулевые элементы.** `scipy.sparse.csr_matrix(X_train_bow)`

In [29]:
from scipy import sparse
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import HashingVectorizer

In [13]:
# Your code here

# Конкатенация
X_train_concat = np.array([' '.join(i) for i in X_train])
X_test_concat = np.array([' '.join(i) for i in X_test])

### Word Vectors (1.5 балл)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах их интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [14]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

--2020-04-24 20:51:10--  https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0x7oxso6x93efzj/ru.tar.gz [following]
--2020-04-24 20:51:10--  https://www.dropbox.com/s/raw/0x7oxso6x93efzj/ru.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc925cb761bd168f7bba8877ea21.dl.dropboxusercontent.com/cd/0/inline/A2ezfbY_3HVyymiv5uJzn1dCkoIsRGvjwxezL0Cx2GWy5YHGyDXjxKVCnto6L3lPMJf65GTlsXT-yw1Y8W9eYctLWP3vMsSToGc3MLJJb1iKVA/file# [following]
--2020-04-24 20:51:11--  https://uc925cb761bd168f7bba8877ea21.dl.dropboxusercontent.com/cd/0/inline/A2ezfbY_3HVyymiv5uJzn1dCkoIsRGvjwxezL0Cx2GWy5YHGyDXjxKVCnto6L3lPMJf65GTlsXT-yw1Y8W9eYctLWP3vMsSToGc3MLJJb1iKVA/file
Resolving uc925cb761bd168f7b

In [23]:
!tar -xzf ru.tar.gz

In [24]:
import gensim
from gensim.models.wrappers import FastText

model = FastText.load_fasttext_format('ru.bin')

In [25]:
# Эмбеддинг предложения -- сумма эмбеддингов токенов


def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """

    embedding_dim = model['кек'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')
    
    for word in sentence.split():
        if word in model:
            features += model[word]
    
    return features

In [26]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

__Задание:__ 
* Обучите логистическую регрессию и SVM на новых признаках
* Оцените качество (accuracy_score)

In [27]:
def items_to_emb(items: np.array) -> np.array:
    return np.array([sentence_embedding(i) for i in tqdm(items)])

In [28]:
# word_vec представление
X_train_bow = items_to_emb(X_train_concat)
X_test_bow = items_to_emb(X_test_concat)

# Проверка
X_train_bow[0]

array([-0.03451314,  0.01240876, -0.01735052,  0.02355238, -0.17378063,
       -0.040433  ,  0.1016985 , -0.10791509,  0.08838575, -0.08215487,
       -0.21942246,  0.12586278, -0.21171936, -0.03043897, -0.12046637,
       -0.19913018, -0.1117378 , -0.04003292, -0.08557754, -0.37242028,
       -0.17062506, -0.05953715, -0.12254356, -0.01909185, -0.08026352,
        0.13826573, -0.14950468,  0.14625254, -0.05017392,  0.14617229,
       -0.02424154, -0.08645173, -0.20779806, -0.31117874,  0.19115713,
        0.03228761,  0.18724765,  0.04775549,  0.03913321, -0.08836071,
        0.14993842, -0.22180234,  0.06136279,  0.04841809, -0.13999987,
        0.03855808,  0.10478982,  0.2772767 , -0.04877075, -0.381868  ,
        0.08603401, -0.35716718,  0.11138823,  0.06087784,  0.02374697,
       -0.16147695,  0.03397164, -0.17635442,  0.0537786 , -0.07753431,
       -0.08055242,  0.26913303, -0.19492012, -0.05666009,  0.01484822,
        0.18831411, -0.1724672 , -0.01974844, -0.03173206,  0.26

In [32]:
len(X_train_bow[0])

300

In [30]:
# Логистическая регрессия
bow_model_lr = LogisticRegression(max_iter=100)
bow_model_lr.fit(X_train_bow, y_train)
acc_lr_bow = accuracy_score(bow_model_lr.predict(X_test_bow), y_test)
print('accuracyLR =', acc_lr_bow)

accuracyLR = 0.4351111111111111


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [31]:
# SVC
bow_model_svc = LinearSVC(max_iter=70)
bow_model_svc.fit(X_train_bow, y_train)
acc_svc_bow = accuracy_score(bow_model_svc.predict(X_test_bow), y_test)
print('accuracySVC =', acc_svc_bow)

accuracySVC = 0.43433333333333335


/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Качество моделей получилось весьма низким (около 43% верных ответов) в сравнении с другими методами. Видимо сказывается очень ограниченное (малое) количество признаков -- 300.

### Что дальше?

Решение каждого пункта 2 балла:



1. N-Gram модели текстовой классификации
    * Признаки - mystem
    * n-gramm'ы - несколько слов подряд объединяются в один токен
    * Модели - только логистическая регрессия
    * Настоятельно рекомендуется использовать sparse-матрицы
    

2. Использовать Vowpal Wabbit вместо sklearn
    * Признаки - обычный BoW title+description 
    * Главный вызов - заставить работать библиотеку и подготовить признаки

3. Другие способы лемматизации (pymorphy2, spaCy)


* Снабжайте код пояснениями;
* Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.